### Imports

In [ ]:
!pip install --quiet tabpfn

In [4]:
import requests
import pandas as pd
!pip install --quiet tabpfn
from tabpfn import TabPFNClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import torch
import torch
import random
import math, time, os
import pandas as pd
from matplotlib import pyplot as plt
from tqdm import tqdm
from pyarrow.parquet import ParquetFile
import pyarrow as pa
import numpy as np
import numpy as np
import torch
from torch import nn
from torch import optim
from IPython import display
import psutil, gc
# utils.hide_toggle('Imports 2')
from torch.utils.data import Dataset
import sklearn.datasets as skds
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
import math
from numpy import sin, pi
from matplotlib import pyplot as plt
from torchvision.datasets import MNIST
import pathlib
from PIL import Image
from PIL.ImageOps import invert
from IPython import display
from time import sleep
import pickle
from matplotlib import pyplot as plt

### Low Noise data (target variable)

In [40]:
df = pd.read_csv('df_synA_train_shuffled.csv',on_bad_lines='skip')
df = df.dropna()

In [41]:
class_values=list(df.target_5_val.unique())
class_values

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[0.0, 0.25, 0.75, 0.5, 1.0]

In [42]:
def encode(v,class_values=class_values):
    return class_values.index(v)

In [43]:
df['era_label']=df['target_5_val'].apply(encode)
df['era_label'].unique()
df=df.sample(frac=1.0)
df

,Open_n_val,High_n_val,Low_n_val,Close_n_val,Volume_n_val,SMA_10_val,SMA_20_val,CMO_14_val,High_n-Low_n_val,Open_n-Close_n_val,...,Close_n_slope_3_changelen_val,Close_n_slope_5_changelen_val,Close_n_slope_10_changelen_val,row_num,day,era,target_10_val,target_5_val,data_type,era_label
68752,0.25,0.50,0.25,0.25,0.0,0.50,0.25,0.75,1.00,0.50,...,0.25,0.25,0.75,93,389,22,0.25,0.25,train,1
178821,0.50,0.50,0.50,0.50,0.0,0.50,0.75,0.00,1.00,0.50,...,0.25,0.50,1.00,85,460,0,0.75,0.50,train,3
7111,0.00,0.00,0.25,0.25,0.0,0.00,0.00,0.50,0.00,0.00,...,0.50,0.75,0.75,138,526,22,0.25,0.25,train,1
195439,0.50,0.50,0.50,0.50,0.0,0.50,0.75,0.00,1.00,0.50,...,0.25,0.75,0.50,88,460,1,0.75,0.75,train,2
10151,1.00,1.00,1.00,1.00,0.0,1.00,0.75,0.75,0.75,0.75,...,0.25,1.00,0.50,118,417,3,0.75,0.50,train,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194523,0.50,0.50,0.50,0.50,0.0,0.75,0.75,0.25,0.75,0.00,...,0.75,0.75,0.75,77,403,0,0.75,0.25,train,1
206880,1.00,1.00,1.00,1.00,0.0,1.00,0.75,1.00,0.50,0.00,...,0.00,0.00,1.00,109,548,17,0.00,0.00,train,0
46593,1.00,1.00,1.00,1.00,0.0,1.00,1.00,0.75,0.75,0.50,...,0.00,0.50,0.50,135,415,3,0.75,0.75,train,2
263444,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.25,0.75,0.75,...,0.50,0.75,0.50,123,541,23,0.25,0.25,train,1


In [44]:
len(df)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


312000

In [45]:
df = df[:1000]

In [46]:
df_train=df.iloc[0:(len(df)*7)//10]
df_test = df.iloc[(len(df)*7)//10:]

In [47]:
x_train=df_train.iloc[:,0:-6].values
y_train=df_train.iloc[:,-1].values
x_test=df_test.iloc[:,0:-6].values
y_test=df_test.iloc[:,-1].values

In [48]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((700, 25), (700,), (300, 25), (300,))

In [39]:
classifier = TabPFNClassifier(device='cpu', N_ensemble_configurations=32)

classifier.fit(x_train, y_train)
y_eval, p_eval = classifier.predict(x_test, return_winning_probability=True)

print('Accuracy', accuracy_score(y_test, y_eval))

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Accuracy 0.7333333333333333


### Low Noise Data (era variable)

In [5]:
df = pd.read_csv('df_synA_train_shuffled.csv',on_bad_lines='skip')
df = df.dropna()
class_values=list(df.era.unique())
def encode(v,class_values=class_values):
    return class_values.index(v)
df['era_label']=df['era'].apply(encode)
df['era_label'].unique()
df=df.sample(frac=1.0)
df = df[:1000]
df_train=df.iloc[0:(len(df)*7)//10]
df_test = df.iloc[(len(df)*7)//10:]
x_train=df_train.iloc[:,0:-6].values
y_train=df_train.iloc[:,-1].values
x_test=df_test.iloc[:,0:-6].values
y_test=df_test.iloc[:,-1].values

#### Grouping rarest clases (to overcome TabPFN's restriciton on 10 classes)

In [35]:
import pandas as pd
from sklearn.metrics import accuracy_score

# Load and preprocess the dataset
df = pd.read_csv('df_synA_train_shuffled.csv', on_bad_lines='skip')
df = df.dropna()

# Identifying the four rarest classes
class_counts = df['era'].value_counts()
rarest_classes = class_counts.nsmallest(4).index.tolist()

# Group the four rarest classes into two
group_1 = rarest_classes[:2]
group_2 = rarest_classes[2:]

def group_rare_classes(row):
    if row in group_1:
        return 'group_1'
    elif row in group_2:
        return 'group_2'
    else:
        return row

df['era'] = df['era'].apply(group_rare_classes)

# Updating class_values and the encode function
class_values = list(df['era'].unique())

def encode(v, class_values=class_values):
    return class_values.index(v)

df['era_label'] = df['era'].apply(encode)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [36]:
# Continue with the rest of your process
df = df.sample(frac=1.0)
df = df[:1000]
df_train = df.iloc[0:(len(df) * 7) // 10]
df_test = df.iloc[(len(df) * 7) // 10:]
x_train = df_train.iloc[:, 0:-6].values
y_train = df_train.iloc[:, -1].values
x_test = df_test.iloc[:, 0:-6].values
y_test = df_test.iloc[:, -1].values

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [37]:
classifier = TabPFNClassifier(device='cpu', N_ensemble_configurations=32)
classifier.fit(x_train, y_train)
y_eval, p_eval = classifier.predict(x_test, return_winning_probability=True)

print('Accuracy:', accuracy_score(y_test, y_eval))

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Accuracy: 0.6466666666666666


#### Multiple Classifiers (to overcome TabPFN's restriciton on 10 classes)

In [39]:
import pandas as pd
from sklearn.metrics import accuracy_score

# Load the dataset
df = pd.read_csv('df_synA_train_shuffled.csv', on_bad_lines='skip')
df = df.dropna()

# Ensure the classes are evenly distributed
class_values = list(df.era.unique())

# Split the classes into two groups
group1_classes = class_values[:6]
group2_classes = class_values[6:]

# Create two subsets of the data
df_group1 = df[df.era.isin(group1_classes)].sample(frac=1.0).iloc[:1000]
df_group2 = df[df.era.isin(group2_classes)].sample(frac=1.0).iloc[:1000]

# Encode the classes for each group
def encode(v, class_values):
    return class_values.index(v)

df_group1['era_label'] = df_group1['era'].apply(lambda x: encode(x, group1_classes))
df_group2['era_label'] = df_group2['era'].apply(lambda x: encode(x, group2_classes))

# Split each group into training and testing sets
def split_data(df):
    df_train = df.iloc[0:(len(df)*7)//10]
    df_test = df.iloc[(len(df)*7)//10:]
    x_train = df_train.iloc[:, 0:-6].values
    y_train = df_train.iloc[:, -1].values
    x_test = df_test.iloc[:, 0:-6].values
    y_test = df_test.iloc[:, -1].values
    return x_train, y_train, x_test, y_test

x_train1, y_train1, x_test1, y_test1 = split_data(df_group1)
x_train2, y_train2, x_test2, y_test2 = split_data(df_group2)

# Initialize the classifiers
classifier1 = TabPFNClassifier(device='cpu', N_ensemble_configurations=32)
classifier2 = TabPFNClassifier(device='cpu', N_ensemble_configurations=32)

# Train the classifiers
classifier1.fit(x_train1, y_train1)
classifier2.fit(x_train2, y_train2)

<frozen importlib._bootstrap>:914: ImportWarning: _PyDrive2ImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _GenerativeAIImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _OpenCVImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: APICoreClientInfoImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _AltairImportHook.find_spec() not found; falling back to find_module()


TabPFNClassifier(N_ensemble_configurations=32)

In [40]:
# Predict with both classifiers
y_pred1 = classifier1.predict(x_test1)
y_pred2 = classifier2.predict(x_test2)
y_combined_pred = np.concatenate([y_pred1, y_pred2])
y_combined_test = np.concatenate([y_test1,y_test2])
accuracy = accuracy_score(y_combined_test, y_combined_pred)
print('Combined Model Accuracy:', accuracy)

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Combined Model Accuracy: 0.7933333333333333


### Ensemble of models for the method with 2 classifiers (trained with 80k rows)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Load the dataset
df = pd.read_csv('df_synA_train_shuffled.csv', on_bad_lines='skip')
df = df.dropna()

# Define the class values and split them into two groups
class_values = list(df.era.unique())
group1_classes = class_values[:6]
group2_classes = class_values[6:]

# Function to encode class labels
def encode(v, class_values):
    return class_values.index(v)

# Initialize a list to store the classifiers and their corresponding class labels
ensemble_classifiers = []

# Create multiple subsets (e.g., 156 as you have 312,000 rows and want 2,000 per subset)
n_subsets = 40
for _ in range(n_subsets):
    # Sample 2000 rows for this iteration
    subset = df.sample(n=2000, replace=False)

    # Create subsets for each group of classes
    df_group1 = subset[subset.era.isin(group1_classes)]
    df_group2 = subset[subset.era.isin(group2_classes)]

    # Apply encoding
    df_group1['era_label'] = df_group1['era'].apply(lambda x: encode(x, group1_classes))
    df_group2['era_label'] = df_group2['era'].apply(lambda x: encode(x, group2_classes))

    # Split data into train and test sets
    x_train1, x_test1, y_train1, y_test1 = train_test_split(df_group1.iloc[:, 0:-6].values, df_group1.iloc[:, -1].values, test_size=0.3)
    x_train2, x_test2, y_train2, y_test2 = train_test_split(df_group2.iloc[:, 0:-6].values, df_group2.iloc[:, -1].values, test_size=0.3)

    # Train classifiers
    classifier1 = TabPFNClassifier(device='cpu', N_ensemble_configurations=32)
    classifier2 = TabPFNClassifier(device='cpu', N_ensemble_configurations=32)
    classifier1.fit(x_train1, y_train1)
    classifier2.fit(x_train2, y_train2)

    # Store classifiers and their class values
    ensemble_classifiers.append(((classifier1, group1_classes), (classifier2, group2_classes)))

# Now, you need to define a method for combining predictions from all classifiers for a test set.
# This method depends on how you want to aggregate predictions - through voting, averaging probabilities, etc.
# Example: Majority voting or averaging probabilities for each class and then picking the class with the highest score.


In [12]:
# Assume your test set is df_test
x_test = df_test.iloc[:, 0:-6].values
y_test = df_test.iloc[:, -1].values  # Actual labels for evaluating accuracy

# Function to decode prediction back to original class label
def decode(pred, class_values):
    return class_values[pred]

# Collecting predictions from all classifiers
all_predictions = []

for classifiers in tqdm(ensemble_classifiers):
    classifier1, group1_classes = classifiers[0]
    classifier2, group2_classes = classifiers[1]

    # Predict with both classifiers
    y_pred1 = classifier1.predict(x_test)
    y_pred1_decoded = [decode(pred, group1_classes) for pred in y_pred1]

    y_pred2 = classifier2.predict(x_test)
    y_pred2_decoded = [decode(pred, group2_classes) for pred in y_pred2]

    # Combine predictions
    combined_predictions = []
    for p1, p2 in zip(y_pred1_decoded, y_pred2_decoded):
        combined_predictions.append(p1 if p1 in group1_classes else p2)

    all_predictions.append(combined_predictions)

# Majority voting for each data point
final_predictions = []
for i in tqdm(range(len(x_test))):
    # Count votes for each class for this data point across all classifiers
    votes = {}
    for preds in all_predictions:
        pred = preds[i]
        votes[pred] = votes.get(pred, 0) + 1

    # Determine the class with the most votes
    final_prediction = max(votes, key=votes.get)
    final_predictions.append(final_prediction)

# Convert final predictions back to encoded form for accuracy calculation
final_predictions_encoded = [class_values.index(pred) for pred in final_predictions]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
  0%|          | 0/40 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_ree

In [13]:
# Calculate accuracy
accuracy = accuracy_score(y_test, final_predictions_encoded)
print('Ensemble Accuracy:', accuracy)

Ensemble Accuracy: 0.38666666666666666


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
